In [1]:
import pandas as pd
import spacy
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import syllable_tokenize
import re
from spacy.tokens import Doc

In [2]:
df = pd.read_csv('construct_pj.csv')

In [3]:
# ข้อมูลทั้งสองแบบ
list_words = [
    "ซื้อ", "จัดซื้อ", "ซื้อจัดซื้อ",
    "จ้าง", "จ้างจัดจ้าง", "จ้างจ้าง", "จ้างทำของ/จ้างเหมาบริการ",
    "ประกวดราคา", "ประกวด", "ประกวดราคาจ้าง",
    "เช่า",
    "เหมา", "เหมาประกอบอาหาร", "เหมาราคา","เหมาบริการ",
    "สอบ", "สอบราคา", "โดยวิธีเฉพาะ", "โดยวิธีเฉพาะเจาะจง"
]

dict_words = {
    "ซื้อ": ["ซื้อ", "จัดซื้อ", "ซื้อจัดซื้อ"],
    "จ้าง": ["จ้าง", "จ้างจัดจ้าง", "จ้างจ้าง"],
    "ประกวด": ["ประกวดราคา", "ประกวด", "ประกวดราคาจ้าง"],
    "เช่า": ["เช่า"],
    "เหมา": ["เหมา", "เหมาราคา"],
    "สอบ": ["สอบ", "สอบราคา"],
    "โดย": ["โดยวิธีเฉพาะ","ด้วยวิธีประกวดราคาอิเล็กทรอนิกส์ (e-bidding)","โดยวิธีคัดเลือก",]
}

# รวมข้อมูลจาก dictionary ไปยัง list และลบคำที่ซ้ำกัน
combined_words = list(set(list_words + [word for group in dict_words.values() for word in group]))

# ฟังก์ชันเพื่อลบคำที่ตรงกับคำใน words_to_remove
def remove_specified_words(text):
    for word in combined_words:
        # ลบคำที่ตรงกับคำที่ต้องการ (ไม่ว่าจะมีคำอื่นต่อท้ายหรือไม่)
        while text.startswith(word):  # ใช้ while เพื่อให้ลบคำที่ตรงซ้ำหลายๆ ครั้ง
            text = text[len(word):].strip()  # ลบคำที่ตรงกับคำที่กำหนด
    return text.strip()  # ลบช่องว่างส่วนเกิน

# ใช้ฟังก์ชันกับคอลัมน์ 'project_name'
df['project_name'] = df['project_name'].apply(remove_specified_words)

# แสดงผลลัพธ์
df['project_name']

0        ก่อสร้างก่อสร้างอาคารศูนย์การแพทย์ เป็นอาคาร ค...
1        ก่อสร้างถนนคอนกรีตสายแยกทางหลวงหมายเลข 347 - บ...
2        ก่อสร้างโรงฝึกกีฬาอเนกประสงค์ 3 ชั้น วิทยาเขตก...
3        แก้ไขคันทางทรุดตัวสาย อย.4009 แยกทางหลวงหมายเล...
4        ซ่อมสร้างผิวทางลาดยางพาราแอสฟัลติกคอนกรีตสาย อ...
                               ...                        
22237    จัดทำโครงสร้างไม้เลื้อยและเทพื้น โดยวิธีเฉพาะเ...
22238    ปรับปรุงซ่อมแซมฝาตะแกรงบ่อพักเส้นหน้าไฟแดงทางเ...
22239    ปรับปรุงบ้านพักครูและบ้านพักภารโรง โดยวิธีเฉพา...
22240    รายงานขอจ้างเหมาปรับปรุงพื้นห้องให้บริการแพทย์...
22241          ซ่อมแซมหลังคาและฝ้าเพดาน โดยวิธีเฉพาะเจาะจง
Name: project_name, Length: 22242, dtype: object

In [16]:
# ฟังก์ชันกรองคำที่ไม่จำเป็น
def clean_text(text):

    text = text.strip()

    # ลบรูปแบบที่มีคำบ่งบอกจำนวนตามด้วยตัวเลขและคำต่อท้าย เช่น "จำนวน 1 ระบบ" หรือ "ระยะทาง 1.105 กม."
    text = re.sub(r'(จำนวน|ระยะทาง|ระยะห่าง|ไม่น้อยกว่า|ไม่ต่ำกว่า|รวม|พื้นที่|พื้นที่ไม่น้อยกว่า|ความยาวไม่น้อยกว่า|ครั้งละ|ขนาด|กว้าง|ยาว|สูง|หนา|เฉลี่ย|x|วงเงิน|รหัสสินทรัพย์|SERIAL NO\.|ครั้ง|ครั้งที่|หมายเลขทะเบียน|กต|ที่|สมัยที่|ม้วนละ|จำนวนเงิน)\s*[\d,]+(\.\d+)?\s*(คัน|ระบบ|กม\.|ตร\.ม\.|ซม\.|มม\.|กิโลเมตร|ML|ซีซี|เมตร|รายการ|ชิ้น|กระบอก|บาท|เล่ม|ผืน|ปทุมธานี|ประตู|นิ้ว|เส้น|BTUH|ชุด|เควี|แอมป์|ม้วน|ตารางเมตร|ถุง|ที่น่ัง|งาน|วัน|แห่ง|หลัง|ต้น|ตัว|เครื่อง|ชั่วโมง|บีทียู|เที่ยว|ขนาด|ขวด|หลอด)?', '', text)

    # ลบรูปแบบที่มีขนาดที่ใช้ x เป็นตัวคูณ เช่น "20x20x30"
    #text = re.sub(r'\b\d+(\.\d+)?(x\d+(\.\d+)?)+\b', '', text)

    # ลบเฉพาะ "- บ้าน", "- บ.", "- เขต", "- ที่อยู่" โดยไม่ลบข้อความอื่นๆ
    #text = re.sub(r'\s*-\s*(บ้าน|บ\.|เขต|ที่อยู่|คลอง|เลียบคลอง).*', '', text)

    # # ลบข้อความในวงเล็บที่มีคำบ่งบอกจำนวน เช่น (ตอนที่...), (หมายเลข...), (เลขครุภัณฑ์...)
    #text = re.sub(r'\((ตอนที่|หมายเลข|เลขครุภัณฑ์)[^\)]*\)', '', text)

    # ลบข้อความที่เกี่ยวกับปีงบประมาณและเดือน/ปี เช่น "ประจำปีงบประมาณ พ.ศ.2561", "ประจำเดือนตุลาคม 2560"
    text = re.sub(r"(ปี|ประจำปี|ประจำปีงบประมาณ|ประจำเดือน|ปีการศึกษา)\s.*$", "", text)

    # ลบตัวเลขทะเบียนที่มี "-" เช่น "82-7049"
    #text = re.sub(r'\b\d{2}-\d{4}\b', '', text).strip()
    #text = re.sub(r'\b\d+-\d+\b', '', text)
    #text = re.sub(r'\b\d-\d\b-\d\b', '', text)

    # # ลบข้อมูลที่ขึ้นต้นด้วยคำว่า "หมู่ที่" หรือ "เลขที่" ตามด้วยตัวเลข เช่น "หมู่ที่ 3,12,13"
    text = re.sub(r'\b(หมู่ที่|เลขที่)\s*[\d,]+\b', '', text)

    # # ลบข้อความที่เกี่ยวกับวัน, เดือน, ปี เช่น ศุกร์, 2 กันยายน
    text = re.sub(r'(วัน\s*(จันทร์|อังคาร|พุธ|พฤหัสบดี|ศุกร์|เสาร์|อาทิตย์))|(\d{1,2}\s*(มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน|ตุลาคม|พฤศจิกายน|ธันวาคม))|(\d{1,2}\s*(มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน|ตุลาคม|พฤศจิกายน|ธันวาคม)\s*\d{4})', '', text)

    # # ลบข้อความที่เกี่ยวกับวัน, เดือน, ปี
    text = re.sub(r'(วันที่|วันที่\s*\d+\s*(กันยายน|ตุลาคม|พฤศจิกายน|ธันวาคม|มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน)\s*\d{4}|พ.ศ\.\s*\d{4}|ประจำปี\s*งบประมาณ\s*\d{4}|ในวัน\s*(วันศุกร์|วันเสาร์|วันอาทิตย์|วันจันทร์|วันอังคาร|วันพุธ|วันพฤหัสบดี)|\s*\d+\s*(มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน)\s*\d{4}|ครั้งที่\s*\d+\s*ประจำปี\s*\d{4})', '', text)

    # # ลบตัวเลขและเดือนที่อยู่ติดกัน เช่น "2 กรกฎาคม"
    text = re.sub(r'\b\d{1,2}\s*(มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน|ตุลาคม|พฤศจิกายน|ธันวาคม)\b', '', text)

    # # ลบข้อความที่เกี่ยวกับที่อยู่ เช่น หมู่ที่, ตำบล, อำเภอ, จังหวัด
    text = re.sub(r'(หมู่ที่\s*\d+|ตำบล\s*[^ ]+|อำเภอ\s*[^ ]+|จังหวัด\s*[^ ]+|ซอย\s*[^ ]+|สมัย\s*[^ ]+|ช่วง\s*[^ ])', '', text)

    # # ลบข้อความเกี่ยวกับยี่ห้อและรุ่น เช่น "ยี่ห้อ" หรือ "รุ่น"
    text = re.sub(r'(ยี่ห้อ\s*[^ ]+|รุ่น\s*[^ ]+)', '', text)

    # ลบข้อความที่เกี่ยวกับหมายเลขทะเบียน, ยี่ห้อ, รหัสสินทรัพย์
    text = re.sub(r'(หมายเลขทะเบียน|ทะเบียน|ยี่ห้อ|รุ่น|หมู่|ปีที่|ฉบับที่|รหัสสินทรัพย์|หมายเลขครุภัณฑ์)\s*[^ ]+', '', text)

    # ลบทะเบียนรถรูปแบบ "กฉ 2767" หรือ "กม 1234"
    text = re.sub(r'\b[ก-ฮ]{1,2}\s?\d{4}\b', '', text)

    # ลบข้อความที่อยู่ในรูปแบบ "(93.14.18.00)" หรือคล้ายกัน
    text = re.sub(r'\(\d{2}\.\d{2}\.\d{2}\.\d{2}\)', '', text)

    # # ลบข้อความที่เกี่ยวกับหมายเลขทะเบียน (มีรูปแบบตัวเลขหรือรหัส)
    text = re.sub(r'\d{1,4}-\d{1,4}|\d{1,4} [A-Z]{1,2}-\d{1,4}|\([A-Za-z0-9\-]+\)', '', text)

    # # ลบตัวเลขที่อยู่ในวงเล็บเฉพาะที่มีตัวเลขภายในวงเล็บ
    text = re.sub(r'\(\d[\d\s]*\)', '', text)

    # # ลบตัวเลขที่มี 4 หลักขึ้นไป
    text = re.sub(r'\b\d{4,}\b', '', text)

    # # ลบตัวเลขโดด
    text = re.sub(r'\b\d\b', '', text)

    # # ลบตัวเลขทศนิยม เช่น 2.5, 100.55
    text = re.sub(r'\d+\.\d+', '', text)
    text = re.sub(r'\(\d+\.\d+\)', '', text)

    # # ลบตัวเลขที่มีเครื่องหมาย - เช่น 12-15, 100-200
    text = re.sub(r'\d+-\d+', '', text)
    text = re.sub(r'\(\d+-\d+\)', '', text)


    # # ลบตัวเลขที่มีเครื่องหมายคอมม่า เช่น 2,700
    text = re.sub(r'\d{1,3}(?:,\d{3})*', '', text)

    # ลบหน่วยที่เกี่ยวข้องกับตัวเลข เช่น เมตร, ตารางเมตร
    text = re.sub(r'\d+\s*(รายการ|เมตร|ตารางเมตร|กิโลกรัม|กิโลเมตร|ซีซี|ลิตร|ชิ้น|คัน|ชั่วโมง|วัน|ปี|เดือน|ครั้ง|รอบ|ลบ\.ม\.|ม\.)', '', text)

    #ข้างหน้าจำนวนข้างหลังเป็นอะไรก้ได้
    text = re.sub(r'(จำนวน).*', "", text)

    #ลบโดยวิธี ด้วยวิธี ข้างหลังเป็นอะไรก็ได้
    text = re.sub(r'(โดยวิธี|ด้วยวิธี).*', "", text)

    # ตัดคำจากข้อความ
    #words = word_tokenize(text, engine="newmm")

    # ลบวงเล็บที่ว่างหรือวงเล็บที่ไม่มีเนื้อหา
    text = re.sub(r"[()\[\]{}]", "", text)

    #ลบทุกช่องว่าง
    text = re.sub(r"\s+", "", text)

    #ลบลูกน้ำ
    text = re.sub(r",", "", text)

    #ลบภาษาอังกฤษ
    text = re.sub(r"^[a-zA-Z].*$", "", text)

    #ลบภาษาอังกฤษที่ผสมในประโยค
    text = re.sub(r"[a-zA-Z]", "", text)

    # ลบคำที่ขึ้นต้นด้วยตัวเลขตามด้วย "รายการ"
    text = re.sub(r"\d+รายการ", "", text)

    # ลบข้อความหลังคำว่า "สาย"
    text = re.sub(r"สาย.*$", "", text)

    # ลบคำว่า "ราคาจ้าง" และ "ราคาซื้อ"
    text = re.sub(r"ราคาจ้าง|ราคาซื้อ|ก่อสร้าง|จ้างก่อสร้าง", "", text)

    # ลบข้อความที่ระบุ ต. อ. และจังหวัด
    #text = re.sub(r"ต\..*?", "", text)  # ลบตั้งแต่ ต. ตามด้วยอะไรก็ได้
    #text = re.sub(r"อ\..*?", "", text)
    #text = re.sub(r"จ\..*?", "", text)

    # ใช้ regex ลบ . + -
    text = re.sub(r'[.+-]', "", text)

    # ลบข้อความที่มี กม.+-กม.+ หรือรูปแบบใกล้เคียง
    text = re.sub(r"กม\.\+?-กม\.\+?.*", "", text)

    #ลบ /
    text = re.sub(r"/", "", text)

    text = text.replace("คสล", "คอนกรีตเสริมเหล็ก")

    # รวมคำที่เหลือกลับเป็นข้อความ
    return text.strip()

# ใช้ฟังก์ชันกับคอลัมน์ 'project_name'

df['project_name'] = df['project_name'].apply(clean_text) #regex หลังตัดพวกจ้างเหมาออก

In [17]:
df['project_name'].to_csv('view_pasi.csv')